In [18]:
import openai
import pinecone
import pandas as pd
from uuid import uuid4
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

## Produce embeddings and package vectors

In [19]:
openai.api_key = "--"
pinecone.init(api_key = "--", environment = 'us-east1-gcp')
vdb = pinecone.Index("testbed")

In [20]:
df = pd.read_csv("clean plant DB.csv")

def gpt3_embedding(content, engine='text-embedding-ada-002'):
    content = content.encode(encoding='ASCII',errors='ignore').decode()  # fix any UNICODE errors
    response = openai.Embedding.create(input=content,engine=engine)
    vector = response['data'][0]['embedding']  # this is a normal list
    return vector
    
payload = list()

for i, row in df.iterrows():
    text = row[0]+row[1]+row[-1]
    vector = gpt3_embedding(text)
    unique_id = str(uuid4())
    payload.append((unique_id, list(vector), {"type": "Plants", "latin_name": row[0], "common_name": row[1],"text":text}))
    # break

## Setup and upsert to pinecone VDB

In [21]:
vdb.upsert(payload, namespace="casia_kb")

{'upserted_count': 130}

## Plant 411 data

In [22]:
pdf411 = pd.read_csv("plant_411_posts.csv")

pdf411

,Unnamed: 0,base_url,title,post_link,post_date,tags,post_content
0,0,https://www.houseplant411.com/houseplant/page/1,8 Wandering Jew Plant Varieties You Need in Yo...,https://www.houseplant411.com/houseplant/8-wan...,NaN,NaN,Indoor plants are a great and affordable way t...
1,1,https://www.houseplant411.com/houseplant/page/1,Echeveria Plant,https://www.houseplant411.com/houseplant/echev...,NaN,NaN,An Echeveria is a succulent plant in the Crass...
2,2,https://www.houseplant411.com/houseplant/page/1,Staghorn Fern,https://www.houseplant411.com/houseplant/stagh...,NaN,NaN,"A staghorn fern, also called an elkhorn fern, ..."
3,3,https://www.houseplant411.com/houseplant/page/1,Persian Shield Plant,https://www.houseplant411.com/houseplant/persi...,NaN,NaN,"A Persian Shield plant, sometimes called a Roy..."
4,4,https://www.houseplant411.com/houseplant/page/1,Purple Waffle Plant,https://www.houseplant411.com/houseplant/purpl...,NaN,NaN,"A Purple Waffle plant, also called Metal Leaf ..."
...,...,...,...,...,...,...,...
137,137,https://www.houseplant411.com/houseplant/page/14,Parlor Palm,https://www.houseplant411.com/houseplant/parlo...,NaN,NaN,Chamadorea elegans (Neanthe bella or Neanthe ...
138,138,https://www.houseplant411.com/houseplant/page/14,Baby’s Tears Plant,https://www.houseplant411.com/houseplant/babys...,NaN,NaN,"A Baby’s Tears plant, Helxine soleirolii, is n..."
139,139,https://www.houseplant411.com/houseplant/page/14,Zamioculcas Zamiifolia-ZZ Plant,https://www.houseplant411.com/houseplant/zz-pl...,NaN,NaN,"A ZZ plant, Zamioculcas zamiifolia, has many n..."
140,140,https://www.houseplant411.com/houseplant/page/15,Shamrock Plant,https://www.houseplant411.com/houseplant/shamr...,NaN,NaN,A Shamrock plant is a member of the Oxalis gen...


In [27]:
payload411 = list()

for i, row in pdf411.iterrows():
    text = row[2]+row[-1]
    vector = gpt3_embedding(text)
    unique_id = str(uuid4())
    payload411.append((unique_id, list(vector), {"type": "Plants", "common_name": row[2],"text":text}))
    # break

In [28]:
vdb.upsert(payload411, namespace="casia_kb")

{'upserted_count': 142}